# BlueMath-tk Model Wrapper Tutorial: Parametric Study Setup

This notebook demonstrates how to use BlueMath-tk's Model Wrapper system to set up and execute parametric studies with numerical models. We'll use a `DummyModelWrapper` as an example, but the same principles apply to real model wrappers like SWASH, SWAN, Delft3D, etc.

## 1. Define Parameter Space

The first step is defining the parameter space for our study. Each parameter can have multiple values that we want to test in different combinations.

In [11]:
metamodel_parameters = {
    "manning": [1, 2, 3],  # Manning roughness coefficient values
    "alpha": [4, 5, 6],  # Alpha parameter values
    "dy": [0.1, 0.2],  # Grid spacing values
}
metamodel_parameters

{'manning': [1, 2, 3], 'alpha': [4, 5, 6], 'dy': [0.1, 0.2]}

## 2. Set Up Directory Structure and Fixed Parameters

Define where templates are stored, where outputs should go, and any parameters that remain constant across all model runs.

In [12]:
import os

root_dir = os.getcwd()  # Use current working directory as root directory
templates_dir = os.path.join(
    root_dir, "DummyTemplates"
)  # Directory containing input file templates
output_dir = os.path.join(root_dir, "DummyOutput")  # Directory for model outputs
fixed_parameters = {"advection": 1}  # Parameters that don't vary

print(f"Templates directory: {templates_dir}")
print(f"Output directory: {output_dir}")
print(f"Fixed parameters: {fixed_parameters}")

Templates directory: /vols/abedul/home/grupos/valvanuz/HySwash/BlueMath/toolkit/wrappers/DummyTemplates
Output directory: /vols/abedul/home/grupos/valvanuz/HySwash/BlueMath/toolkit/wrappers/DummyOutput
Fixed parameters: {'advection': 1}


## 3. Initialize the Model Wrapper

Create an instance of the model wrapper with our configuration. The wrapper will handle case generation, template rendering, and model execution.

In [13]:
from bluemath_tk.wrappers._base_wrappers import DummyModelWrapper

class ChangedDummyModelWrapper(DummyModelWrapper):

    def build_case(self, case_dir: str, case_context: dict):
        """
        YOUR build_case() method.
        """

        case_context["nmax"] = 0.5 * case_context["dy"]

    def postprocess_case(self, **kwargs) -> None:
        """
        YOUR postprocess_case() method.
        """

        pass

In [ ]:
hurrywaves_model = DummyModelWrapper(
    templates_dir=templates_dir,
    metamodel_parameters=metamodel_parameters,
    fixed_parameters=fixed_parameters,
    output_dir=output_dir,
)

hurrywaves_model.build_cases(
    mode="all_combinations",
    cases_name_format=lambda ctx: f"M_{ctx.get('manning')}_A_{ctx.get('alpha')}_DY_{ctx.get('dy')}",
)


Display the first 5 case directories created for each case generated by the parametric model.

In [25]:
hurrywaves_model.cases_dirs[:5]

# Uncomment if you want to see the id assigned to each case context
# dict(enumerate(hurrywaves_model.cases_dirs[:5])) # Display the first 5 case directories for verification

['/vols/abedul/home/grupos/valvanuz/HySwash/BlueMath/toolkit/wrappers/DummyOutput/M_1_A_4_DY_0.1',
 '/vols/abedul/home/grupos/valvanuz/HySwash/BlueMath/toolkit/wrappers/DummyOutput/M_1_A_4_DY_0.2',
 '/vols/abedul/home/grupos/valvanuz/HySwash/BlueMath/toolkit/wrappers/DummyOutput/M_1_A_5_DY_0.1',
 '/vols/abedul/home/grupos/valvanuz/HySwash/BlueMath/toolkit/wrappers/DummyOutput/M_1_A_5_DY_0.2',
 '/vols/abedul/home/grupos/valvanuz/HySwash/BlueMath/toolkit/wrappers/DummyOutput/M_1_A_6_DY_0.1']

hurrywaves_model.cases_context stores a list of dictionaries, where each dictionary represents the set of parameters (both variable and fixed) used to generate a specific case in the parametric study. This allows easy access to the parameter values for each generated case.

In [26]:
hurrywaves_model.cases_context[:5]

# Uncomment if you want to see the id assigned to each case context
# dict(enumerate(hurrywaves_model.cases_context[:5]))

[{'manning': 1, 'alpha': 4, 'dy': 0.1, 'case_num': 0, 'advection': 1},
 {'manning': 1, 'alpha': 4, 'dy': 0.2, 'case_num': 1, 'advection': 1},
 {'manning': 1, 'alpha': 5, 'dy': 0.1, 'case_num': 2, 'advection': 1},
 {'manning': 1, 'alpha': 5, 'dy': 0.2, 'case_num': 3, 'advection': 1},
 {'manning': 1, 'alpha': 6, 'dy': 0.1, 'case_num': 4, 'advection': 1}]

## 4. Execute Model Runs in Bulk

The `run_cases_bulk()` method executes all cases using a single command. Here we're using SLURM's job array feature to run cases in parallel on a cluster.

In [17]:
# Execute all cases using SLURM job arrays
# hurrywaves_model.run_cases_bulk(launcher="sbatch --array=0-799 launcher.sh")
# hurrywaves_model.run_cases(launcher="sfincs")

print("Bulk execution command submitted!")

Bulk execution command submitted!
